In [ ]:
%%capture
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd

## Scale Invariant Block

In [ ]:
def sic_block(input_tensor, filters, strides, padding, activation):
    def variant_dims(size):
        n_variants = ceil(log(ceil(sqrt(size)), pi))
        return list(map(lambda x: 3**x, range(1, n_variants+1)))

    def make_layer(size):
        kwargs = dict(strides=strides, padding=padding)
        return layers.Conv2D(filters, size, **kwargs)

    size = min(input_tensor.shape[1:-1])
    variants = variant_dims(size)
    conv_layers = map(make_layer, variants)
    conv_outputs = list(map(lambda x: x(input_tensor), conv_layers))
    merged = tf.concat(conv_outputs, axis=-1)
    normalized = layers.BatchNormalization()(merged)
    output = layers.Activation(activation)(normalized)

    return output

## SEBlock

In [ ]:
class SEBlock(layers.Layer):
    def __init__(self, filters, ratio=2):
        super(SEBlock, self).__init__()
        self.block = tf.keras.Sequential([
            layers.GlobalAveragePooling2D(), # Squeeze
            layers.Dense(filters//ratio, activation='relu'),
            layers.Dense(filters, activation='sigmoid'), # Excite
            layers.Reshape([1, 1, filters]),
        ])

    def call(self, inputs):
        return inputs * self.block(inputs)
    
    def get_config(self):
        return dict(filters=self.filters, ratio=self.ratio)